# Introduction

Ce notebook permet d'extraire les séquences en nucléotides des protéines sélectionnées auparavant (grâce au notebook1 (lien)) dans un nouveau fichier fasta, afin de pouvoir réaliser un BLAST et identifier les protéines sélectionnées.
Les séquences de toutes les protéines détectées durant l'expérience sont regroupées dans le fichier "Reads.fa" dans le dossier "input", un numéro de locus étant attribué à chaque séquence nucléotidique de protéine. 
La liste des protéines sélectionnées issu du notebook1 est dans le fichier "significant_proteins.csv"

## Charger les librairies nécessaires et créer les dossiers

In [21]:
suppressPackageStartupMessages(library(seqinr))
suppressPackageStartupMessages(library(BiocManager))
suppressPackageStartupMessages(library(Biostrings))


## Importer les fichiers

In [22]:
# Importer le fichier csv des protéines sélectionnées

proteins_of_interest <- read.csv("outputs/proteins_of_interest.csv", sep= ";")

# Ajouter "locus_" devant chaque nombre dans la colonne 'locus' pour que cela 
#coincide avec les noms du fichier FASTA
proteins_of_interest$locus <- paste0("Locus_", proteins_of_interest$locus)

# Vérifier le résultat
head(proteins_of_interest)

# Importer le fichier .FA (pouvant être fourni par l'auteure de l'étude initiale) qui comprend les séquences de toutes les protéines détectées chez les palourdes durant l'expérience
sequences <- readDNAStringSet("input/Reads.fa")

# Afficher les premières séquences pour vérifier
head(sequences)

,X,Protein.Group,Protein.ID,Accession,Significance,Coverage....,X.Peptides,X.Unique,PTM,MAL_pres,⋯,Group.Profile..Ratio.,Avg..Mass,Description,Fold_Change,Log2_Fold_Change,P_value,log10P_value,Significant,Category,locus
,<int>,<int>,<int>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<lgl>,⋯,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1,63,55,|Locus_4313121.p1,"45,04",36,34,34,Carbamidomethylation,TRUE,⋯,1.77:1.00,60127,GENE.Locus_4313121|Locus_4313121.p1 ORF type:complete len:556 (+) score=143.89 Locus_4313121:57-1724(+),"1,76446280991736","0,819229022345896","0,0415500135407299","-1,38142883034766",Significant,Significant and high fold change,Locus_4313121
2,2,86,129,|Locus_2348137.p1,"34,36",38,26,26,Carbamidomethylation,TRUE,⋯,1.57:1.00,39966,GENE.Locus_2348137|Locus_2348137.p1 ORF type:5prime_partial len:366 (+) score=103.29 Locus_2348137:1-1098(+),"1,57317073170732","0,653675250805171","0,00191196305750407","-2,71852050331365",Significant,Significant and high fold change,Locus_2348137
3,3,100,117,|Locus_2044774.p1,200,34,19,19,Carbamidomethylation,TRUE,⋯,3.54:1.00,46033,GENE.Locus_2044774|Locus_2044774.p1 ORF type:complete len:414 (+) score=73.30 Locus_2044774:33-1274(+),"3,53135313531353","1,82022109781556","0,000209327496153221","-3,6791737212872",Significant,Significant and high fold change,Locus_2044774
4,4,245,470,|Locus_7235177.p1,200,35,9,9,Carbamidomethylation; Oxidation (M),TRUE,⋯,3.29:1.00,27282,GENE.Locus_7235177|Locus_7235177.p1 ORF type:5prime_partial len:231 (+) score=27.98 Locus_7235177:1-693(+),"3,28193832599119","1,71454812805861","0,0180657459865189","-1,74314410045966",Significant,Significant and high fold change,Locus_7235177
5,5,289,283,|Locus_4823168.p1,"97,27",16,8,8,Carbamidomethylation,TRUE,⋯,0.41:1.00,42357,GENE.Locus_4823168|Locus_4823168.p1 ORF type:complete len:388 (+) score=86.53 Locus_4823168:57-1220(+),"0,414166666666667","-1,27171664893359","0,0388613020657051","-1,41048265225715",Significant,Significant and high fold change,Locus_4823168
6,6,25,10,|Locus_381335.p1,"31,78",52,56,7,Carbamidomethylation; Oxidation (M),TRUE,⋯,1.60:1.00,72615,GENE.Locus_381335|Locus_381335.p1 ORF type:complete len:645 (+) score=138.43 Locus_381335:155-2089(+),"1,6017316017316","0,679632419100255","0,0378345848963259","-1,4221110265497",Significant,Significant and high fold change,Locus_381335


DNAStringSet object of length 6:
    width seq                                               names               
[1]  6763 CACATGCCGGTTTTAAAACTGTC...GGAACAGGAACCGGAACGGGTGG Locus_6941
[2]  1888 GAGAAAATGCCCGTAAATAAACA...TGGGAACGCCCCTGGCGGGCGGG Locus_11305
[3]  2460 GGGGCAAATTTCGTGAGTCGGTC...GTATTTAGTGGTCTCAAATTTCC Locus_11646
[4]  3677 GCCCTTCGATAAAAATGAAAATC...AAGCAATCAATTCACACACAGGA Locus_11693
[5]  1695 GGCTGCAGGATTCTGAAGTGAGT...CACGACTTTTCTTCCAAAATGTC Locus_13115
[6]  2088 GTGAATATTAGCGTACAACCATC...CTAGATAAATAATAAAACACGAC Locus_13145

## Transformer le format DNAstringset en dataframe pour les prochaines étapes

In [24]:
##convertir le fasta en dataframe 
fasta_df <- data.frame (
  ID = names (sequences),
  Sequence = as.character(sequences),
  stringsAsFactors = FALSE
  
)

## Extraire les séquences des 49 protéines en les sélectionnant par leur locus

In [25]:
# Effectuer la jointure (inner join) entre fasta_df et proteins_of_interest
result_df <- merge(fasta_df, proteins_of_interest, by.x = "ID", by.y = "locus", all.x = FALSE)

# Sélectionner seulement les colonnes ID et Sequence
result_df <- result_df[, c("ID", "Sequence")]



## Enregistrer les séquences qui nous intéressent dans un nouveau fichier fasta

In [26]:
# Il faut reconvertir le tableau en fasta pour pouvoir l'enregistrer au bon format
# Convertir le dataframe en un objet DNAStringSet
sequences <- DNAStringSet(result_df$Sequence)
names(sequences) <- result_df$ID  # Ajouter les IDs comme noms des séquences

# Enregistrer les séquences dans un fichier FASTA
writeXStringSet(sequences, filepath = "outputs/final_49_seq.fa")

Les séquences des 49 protéines sont maintenant dans le fichier "final_49_seq.fa" dans le dossier outputs. Ce fichier doit être chargé sur le site [NCBI](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PROGRAM=blastn&PAGE_TYPE=BlastSearch&LINK_LOC=blasthome) (visité le 05/12/2024). Voici les étapes : 

1_ Charger le fichier en cliquant sur "Parcourir" 

2_ Cliquer sur BLAST en bas de page 

3_ Une fois la page chargée, cliquer sur "Download ALL" et sélectionner le format txt.

4_Renommer le nouveau fichier "results_blast.txt" et le déposer dans le dossier outputs.

In [28]:
# Charger le package stringr
library(stringr)

#Importer le fichier téléchargé depuis NCBI, normalement disponible dans le dossier outputs (/outputs/results_blast.txt
blast <-readLines("outputs/results_blast.txt")


Le fichier texte obtenu comprend beaucoup d'informations qui en plus ne sont pas présentées sous forme de tableau. Les lignes ci-dessous permettent d'extraire les passages comprenant les loci des séquences, ainsi que la fonction de la protéine qui a obtenu la meilleure correspondance via le blast sur NCBI.

In [30]:
#Chercher les lignes avec le texte "Query"
query_indices <-grep ("Query #", blast)

#Chercher la ligne de la description de la protéine
fonction_indices <-grep ("Alignments:", blast)
results_fonction <- fonction_indices + 2

#Grouper les numéros des lignes qui nous intéressent
all_indices <- sort (c(query_indices, results_fonction[results_fonction <= length(blast)]))

#Créer un tableau qui contient le locus et la description de la protéine
query_description <- data.frame(
  Query = blast [query_indices],
  Description = blast [results_fonction[results_fonction <= length(blast)]],
  stringsAsFactors = FALSE
)

#Extraire les données intéressantes (locus et rôle de la protéine vers des nouvelles colonnes et un nouveau tableau
# Extraire le locus de la colonne 1
query_description$locus <- str_extract(query_description$Query, "Locus_[0-9]+")

# Nettoyer la colonne Description pour garder uniquement la fonction
query_description$Function <- sub(".*: ", "", query_description$Description)       # Retirer tout avant ": "
query_description$Function <- sub("^.*philippinarum ", "", query_description$Function) # Retirer "philippinarum" et tout avant
query_description$Function <- sub(" \\(LOC.*$", "", query_description$Function)    # Retirer tout ce qui suit (LOC...)
query_description$Function <- sub(" LOC\\d+$", "", query_description$Function)     # Retirer le dernier mot LOCXXXXX si présent

# Vérifier le résultat
head(query_description[, c("locus", "Function")])

# Sauvegarder le fichier nettoyé
write.csv(query_description[, c("locus", "Function")], "outputs/cleaned_query_description.csv", row.names = FALSE)


,locus,Function
,<chr>,<chr>
1,Locus_1202748,protein phosphatase 3 catalytic subunit alpha-like
2,Locus_1236277,glycogen debranching enzyme-like
3,Locus_1397283,titin-like
4,Locus_1979842,adenosine deaminase AGSA-like
5,Locus_2044774,"aspartate aminotransferase, cytoplasmic-like"
6,Locus_2057997,thioredoxin domain-containing protein 5-like
